In [21]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [22]:
# 데이터 로드
orders_df = pd.read_csv('Olist/olist_orders_dataset.csv')
items_df = pd.read_csv('Olist/olist_order_items_dataset.csv')
customers_df = pd.read_csv('Olist/olist_customers_dataset.csv')
payments_df = pd.read_csv('Olist/olist_order_payments_dataset.csv')
reviews_df = pd.read_csv('Olist/olist_order_reviews_dataset.csv')
products_df = pd.read_csv('Olist/olist_products_dataset.csv')
sellers_df = pd.read_csv('Olist/olist_sellers_dataset.csv')
translation_df = pd.read_csv('Olist/product_category_name_translation.csv')

## 1. 데이터 결합

In [23]:
# 주문 항목(items) 기준으로 데이터 결합
# items는 각 주문의 상품별 상세 정보를 담고 있으므로 이를 기준으로 결합

# 1. items + orders 결합
master_df = items_df.merge(orders_df, on='order_id', how='left')

# 2. customers 정보 추가
master_df = master_df.merge(customers_df, on='customer_id', how='left')

# 3. products 정보 추가
master_df = master_df.merge(products_df, on='product_id', how='left')

# 4. sellers 정보 추가
master_df = master_df.merge(sellers_df, on='seller_id', how='left')

# 5. 카테고리 번역 정보 추가
master_df = master_df.merge(translation_df, on='product_category_name', how='left')

master_df.head(1).T


,0
order_id,00010242fe8c5a6d1ba2dd792cb16214
order_item_id,1
product_id,4244733e06e7ecb4970a6e2683c13e61
seller_id,48436dade18ac8b2bce089ec2a041202
shipping_limit_date,2017-09-19 09:45:35
price,58.9
freight_value,13.29
customer_id,3ce436f183e68e07877b285a838db11a
order_status,delivered
order_purchase_timestamp,2017-09-13 08:59:02


In [24]:
# 결제 정보 결합 (주문당 여러 결제 가능하므로 집계 필요)
# 주문당 총 결제 금액, 결제 수단별 금액 등 계산
payment_summary = payments_df.groupby('order_id').agg({
    'payment_value': 'sum',
    'payment_type': lambda x: ', '.join(sorted(set(x))), 
    'payment_installments': 'mean'
}).reset_index()
payment_summary.columns = ['order_id', 'total_payment_value', 'main_payment_type', 'avg_installments']

master_df = master_df.merge(payment_summary, on='order_id', how='left')

master_df.head(1).T


,0
order_id,00010242fe8c5a6d1ba2dd792cb16214
order_item_id,1
product_id,4244733e06e7ecb4970a6e2683c13e61
seller_id,48436dade18ac8b2bce089ec2a041202
shipping_limit_date,2017-09-19 09:45:35
price,58.9
freight_value,13.29
customer_id,3ce436f183e68e07877b285a838db11a
order_status,delivered
order_purchase_timestamp,2017-09-13 08:59:02


In [26]:
# 리뷰 정보 결합 -> 최종 만족도 기준
reviews_df['review_answer_timestamp'] = pd.to_datetime(reviews_df['review_answer_timestamp'], errors='coerce')

# review_answer_timestamp 기준으로 내림차순 정렬
reviews_sorted = reviews_df.sort_values('review_answer_timestamp', ascending=False)

# order_id별로 최신 리뷰만 선택
reviews_summary = reviews_sorted.drop_duplicates(subset='order_id', keep='first')[['order_id', 'review_score', 'review_comment_title','review_comment_message']]

master_df = master_df.merge(reviews_summary, on='order_id', how='left')

master_df.head(1).T


,0
order_id,00010242fe8c5a6d1ba2dd792cb16214
order_item_id,1
product_id,4244733e06e7ecb4970a6e2683c13e61
seller_id,48436dade18ac8b2bce089ec2a041202
shipping_limit_date,2017-09-19 09:45:35
price,58.9
freight_value,13.29
customer_id,3ce436f183e68e07877b285a838db11a
order_status,delivered
order_purchase_timestamp,2017-09-13 08:59:02


#### "고객의 최종 만족도"
- 전략: 최신 리뷰(Latest)만 남기기
- 고객이 처음에 1점을 줬다가, 문제가 해결되어 5점으로 수정했을 수도 있고, 반대로 잘 쓰다가 고장 나서 점수를 깎았을 수도 있다. 고객의 현재 마음을 대변하는 것은 가장 마지막에 작성된(또는 수정된) 리뷰이다.
- 방법: review_answer_timestamp를 기준으로 내림차순 정렬 후 첫 번째 것만 선택(drop_duplicates(keep='first')).

#### "리스크 관리 및 이탈 방지"
- 전략: 최저 점수(Min) 선택
- 논리: 부정적인 경험이 긍정적인 경험보다 더 강력하다는 행동 경제학 관점.
 주문한 상품 중 3개는 좋았어도 1개가 최악이었다면, 그 고객은 해당 서비스에 불만을 가질 확률이 높음
- 방법: groupby('order_id')['review_score'].min()

#### "일반적인 경향성 파악"
- 전략: 평균 점수(Mean) 사용
- 논리: 여러 상품을 샀을 때 좋았던 것과 나빴던 것을 상쇄하여 **'평균적인 경험'**으로 치환
단, **3점(보통)과 1점+5점의 평균(3점)을 구분하지 못해 정보가 희석될 수 있음**
- 방법: groupby('order_id')['review_score'].mean()

#### "정보의 풍부함"
- 전략: 텍스트가 있는 리뷰 우선
- 논리: 같은 주문에 대해 하나는 점수만 있고, 하나는 상세한 리뷰글이 있다면 가치 있는 데이터는 글이 있는 쪽
- 방법: review_comment_message가 Null이 아닌 행을 우선순위로 두고 정렬하여 선택.

## 2. 결측치 확인 및 처리


In [28]:
# 결측치 확인
missing_data = master_df.isnull().sum()
missing_percent = (missing_data / len(master_df)) * 100
missing_df = pd.DataFrame({
    '결측치 수': missing_data,
    '결측치 비율(%)': missing_percent
})
missing_df = missing_df[missing_df['결측치 수'] > 0].sort_values('결측치 수', ascending=False)

print(missing_df)


                               결측치 수  결측치 비율(%)
review_comment_title           99235  88.091434
review_comment_message         65252  57.924545
order_delivered_customer_date   2454   2.178429
product_category_name_english   1627   1.444296
product_category_name           1603   1.422992
product_name_lenght             1603   1.422992
product_description_lenght      1603   1.422992
product_photos_qty              1603   1.422992
order_delivered_carrier_date    1194   1.059920
review_score                     942   0.836218
product_height_cm                 18   0.015979
product_width_cm                  18   0.015979
product_length_cm                 18   0.015979
product_weight_g                  18   0.015979
order_approved_at                 15   0.013316
total_payment_value                3   0.002663
main_payment_type                  3   0.002663
avg_installments                   3   0.002663


In [30]:
# 날짜 컬럼 변환 및 결측치 처리
date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date',
                'order_estimated_delivery_date', 'shipping_limit_date']

for col in date_columns:
    if col in master_df.columns:
        master_df[col] = pd.to_datetime(master_df[col], errors='coerce')


In [ ]:
# 제품 카테고리 결측치 처리 (영어 번역이 없으면 포르투갈어 원본 사용)
master_df['product_category_name_english'] = master_df['product_category_name_english'].fillna(
    master_df['product_category_name']
)

# 제품 무게 결측치 처리 (중앙값으로 대체)
if 'product_weight_g' in master_df.columns:
    master_df['product_weight_g'] = master_df['product_weight_g'].fillna(
        master_df['product_weight_g'].median()
    )

# 제품 크기 결측치 처리 (중앙값으로 대체)
size_columns = ['product_length_cm', 'product_height_cm', 'product_width_cm']
for col in size_columns:
    if col in master_df.columns:
        master_df[col] = master_df[col].fillna(master_df[col].median())

print("제품 정보 결측치 처리 완료")
